# Скачиваем и устанавливаем всё необходимое

Удаляем ненужный sample_data

In [ ]:
!rm -rf sample_data

Клонируем наш репозиторий

In [ ]:
!python3 a.py   

python3: can't open file 'a.py': [Errno 2] No such file or directory


In [ ]:
!rm -rf logical_reasoning

In [ ]:
!git clone https://github.com/HSE-projects/logical_reasoning

Cloning into 'logical_reasoning'...
remote: Enumerating objects: 86, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 86 (delta 41), reused 47 (delta 18), pack-reused 4
Unpacking objects: 100% (86/86), done.


Удаляем torchtext так как в модели хотят версию торча 1.6, а она не поддерживается с новым torchtext, torchvision, ...

In [ ]:
!pip uninstall torchtext -y

Found existing installation: torchtext 0.11.0
Uninstalling torchtext-0.11.0:
  Successfully uninstalled torchtext-0.11.0


Устанавливаем всё необходимое

In [ ]:
!pip install -r logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/requirements.txt
!pip install datasets
!pip install gdown

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/requirements.txt'
     |████████████████████████████████| 311 kB 25.3 MB/s 
     |████████████████████████████████| 1.1 MB 57.7 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 133 kB 69.2 MB/s 
     |████████████████████████████████| 243 kB 51.4 MB/s 
     |████████████████████████████████| 271 kB 59.9 MB/s 
     |████████████████████████████████| 160 kB 53.9 MB/s 
     |████████████████████████████████| 144 kB 69.8 MB/s 


Скачиваем оптимальные параметры модели

In [ ]:
# !gdown https://drive.google.com/uc?id=1bJVYekaOVJjI2woxqaqKMhWP0HLau105  # SNLI Base
!gdown https://drive.google.com/uc?id=1AHMGgYFqcr3NmA-py6AMsO83w8FeNSJn  # SNLI Large

Downloading...
From: https://drive.google.com/uc?id=1AHMGgYFqcr3NmA-py6AMsO83w8FeNSJn
To: /content/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt
100% 4.31G/4.31G [01:13<00:00, 58.6MB/s]


In [ ]:
# One of them is absent
!mv /content/epoch=4-valid_loss=-0.6472-valid_acc_end=0.9173.ckpt /content/snli_baseline.ckpt  # SNLI Base
!mv /content/epoch=2-valid_loss=-0.2620-valid_acc_end=0.9223.ckpt /content/snli_baseline.ckpt  # SNLI Large

mv: cannot stat '/content/epoch=4-valid_loss=-0.6472-valid_acc_end=0.9173.ckpt': No such file or directory


Скачиваем модель roberta. Для этого надо установить [git-lfs](https://git-lfs.github.com)

In [ ]:
!apt-get install git-lfs
!git lfs install
# !git clone https://huggingface.co/roberta-base
!git clone https://huggingface.co/roberta-large

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (2,675 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155229 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Error: Failed to call git rev-parse --git-dir --show-toplevel: "fatal: not a git repository (or any of the parent directories): .git\n"
Git LFS initialized.
Cloning into 'roberta-large'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 1

# Тестируем модель

Меняем конфиг, чтобы был `num_labels` = 3

In [ ]:
import json
config = json.load(open('roberta-large/config.json'))
config['num_labels'] = 3
with open('roberta-large/config.json', 'w') as f:
    json.dump(config, f, indent=4)

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.4 MB 26.2 MB/s 
     |████████████████████████████████| 3.3 MB 61.1 MB/s 
     |████████████████████████████████| 895 kB 49.2 MB/s 


In [ ]:
from torch import nn
from transformers.modeling_roberta import RobertaModel, RobertaConfig

from dataset_utils.collate_functions import collate_to_max_length

class ExplainableModel(nn.Module):
    def __init__(self, bert_dir):
        super().__init__()
        self.bert_config = RobertaConfig.from_pretrained(bert_dir, output_hidden_states=False)
        self.intermediate = RobertaModel.from_pretrained(bert_dir)
        self.span_info_collect = SICModel(self.bert_config.hidden_size)
        self.interpretation = InterpretationModel(self.bert_config.hidden_size)
        self.output = nn.Linear(self.bert_config.hidden_size, self.bert_config.num_labels)

    def forward(self, input_ids, start_indexs, end_indexs, span_masks):
        # generate mask
        attention_mask = (input_ids != 1).long()
        # intermediate layer
        hidden_states, first_token = self.intermediate(input_ids, attention_mask=attention_mask)  # output.shape = (bs, length, hidden_size)
        # span info collecting layer(SIC)
        h_ij = self.span_info_collect(hidden_states, start_indexs, end_indexs)
        # interpretation layer
        H, a_ij = self.interpretation(h_ij, span_masks)
        # output layer
        out = self.output(H)
        return out, a_ij

ModuleNotFoundError: ignored

In [ ]:
import pytorch_lightning as pl
pl.metrics.F1()

AttributeError: ignored

## Тестируем модель на [SNLI](https://huggingface.co/datasets/snli)

Да, датасет есть на huggingface, но у авторов есть готовое решение с использованием SNLI, и оно его не использует. Поэтому я скачиваю SNLI с интеренета сам.

Пример объекта из SNLI:

| Столбец | Описание столбца | Значение |
|-------- --|----------------|--------- |
| sentence1 | Предпосылка | An old man with a package poses in front of an advertisement. |
| sentence1_binary_parse | Дерево зависимостей для предпосылки | ( ( ( An ( old man ) ) ( with ( a package ) ) ) ( ( poses ( in ( front ( of ( an advertisement ) ) ) ) ) . ) ) |
| sentence1_parse | Распарсенная предпосылка через Stanford PCFG Parser 3.5.2 | (ROOT (S (NP (NP (DT An) (JJ old) (NN man)) (PP (IN with) (NP (DT a) (NN package)))) (VP <br /> (VBZ poses) (PP (IN in) (NP (NP (NN front)) (PP (IN of) (NP (DT an) (NN advertisement)))))) (. .))) |
| sentence2 | Гипотеза |  A man walks by an ad. |
| sentence2_binary_parse | Дерево зависимостей для гипотезы | ( ( A man ) ( ( walks ( by ( an ad ) ) ) . ) ) |
| sentence2_parse | Распарсенная гипотеза через Stanford PCFG Parser 3.5.2 | (ROOT (S (NP (DT A) (NN man)) (VP (VBZ walks) (PP (IN by) (NP (DT an) (NN ad)))) (. .))) |
| annotator_labels | Ответы пяти mechanical turk workers (зарубежных толокеров) | ["contradiction", "neutral", "contradiction", "contradiction", "contradiction"] |
| gold_label | Ответ на вопрос "Как связаны эти предложения?" | contradiction |
| captionID | ??? |  4791890474.jpg#3 |
| pairID | ??? |  4791890474.jpg#3r1c |

Скачиваем и распаковываем SNLI. Также удаляем ненужные файлы.

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip
!rm -rf __MACOSX
!rm snli_1.0.zip

--2022-01-22 20:05:07--  https://nlp.stanford.edu/projects/snli/snli_1.0.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 94550081 (90M) [application/zip]
Saving to: ‘snli_1.0.zip’

snli_1.0.zip        100%[===================>]  90.17M  3.68MB/s    in 44s     

2022-01-22 20:05:52 (2.03 MB/s) - ‘snli_1.0.zip’ saved [94550081/94550081]

Archive:  snli_1.0.zip
   creating: snli_1.0/
  inflating: snli_1.0/.DS_Store      
   creating: __MACOSX/
   creating: __MACOSX/snli_1.0/
  inflating: __MACOSX/snli_1.0/._.DS_Store  
 extracting: snli_1.0/Icon           
  inflating: __MACOSX/snli_1.0/._Icon  
  inflating: snli_1.0/README.txt     
  inflating: __MACOSX/snli_1.0/._README.txt  
  inflating: snli_1.0/snli_1.0_dev.jsonl  
  inflating: snli_1.0/snli_1.0_dev.txt  
  inflating: snli_1.0/snli_1.0_test.jsonl  
  inflating: snli_1.0/snli_1.0_

Запускаем тестилку:

In [ ]:
!mkdir /content/snli_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--data_dir /content/snli_1.0 \
--task snli \
--checkpoint_path '/content/snli_baseline.ckpt' \
--save_path /content/snli_result/ \
--gpus=0, \
--mode eval
%cd /content

mkdir: cannot create directory ‘/content/snli_result’: File exists
/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1
Testing: 100% 983/983 [04:41<00:00,  4.25it/s]tensor(-0.7510, device='cuda:0') tensor(0.9221, device='cuda:0')
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The testing_epoch_end should not return anything as of 9.1.to log, use self.log(...) or self.write(...) directly in the LightningModule
  warnings.warn(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar

## Тестируем модель на [SICK](https://huggingface.co/datasets/sick)

NLI датасет + gold label + gold score - насколько предложения похожи от 1 до 5. В пары входили: очищенное от времён и прочего предложение + схожее, очищенное + противоположное, очищенное + обратное.

Пример объекта:

| Столбец | Описание столбца | Значение |
|---------|----|----------|
| entailment_AB | Связь из А в В | A_contradicts_B |
| entailment_BA | Связь из В в А | B_neutral_A |
| id | ID объекта | 25 |
| label | Gold label (0 - entailment, 1 - neutral, 2 - contradiction) | 1 |
| relatedness_score | От 1 до 5, насколько предложения похожи | 2.799999952316284 |
| sentence_A | Очищенное предложение А |  Nobody is riding the bicycle on one wheel |
| sentence_A_dataset | Откуда достали А | FLICKR |
| sentence_A_original | Исходное предложение А | A person rides their bicycle onto a rock and balances on one wheel |
| sentence_B  | Очищенное предложение В | A person in a black jacket is doing tricks on a motorbike |
| sentence_B_dataset  | Откуда достали В | FLICKR |
| sentence_B_original  | Исходное предложение В | A person in a black jacket doing tricks on a motorbike. |

Поскольку модель не тестировали на SICK, то её придётся дообучать

In [ ]:
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-base \
--task sick \
--save_path /content/sick_ckpt \
--checkpoint_path '/content/snli_baseline.ckpt' \
--gpus=0,  \
--precision 16 \
--lr=2e-5 \
--batch_size=10 \
--lamb=1.0 \
--workers=1 \
--max_epoch=1
%cd /content

[Errno 2] No such file or directory: '/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain'
/content
python3: can't open file 'trainer.py': [Errno 2] No such file or directory
/content


In [ ]:
!mkdir /content/sick_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--task sick \
--checkpoint_path '/content/sick_ckpt/last.ckpt' \
--save_path /content/sick_result/ \
--gpus=0, \
--mode eval
%cd /content

/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
Traceback (most recent call last):
  File "trainer.py", line 287, in <module>
    main()
  File "trainer.py", line 278, in main
    evaluate(args)
  File "trainer.py", line 265, in evaluate
    checkpoint = torch.load(args.checkpoint_path, map_location=torch.device('cpu'))
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 571, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 229, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 210, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/sick_ckpt/last.ckpt'
/content


## Тестируем модель на [ANLI](https://huggingface.co/datasets/anli)

Стоит сразу заметить, что ANLI создавал Facebook для ML контеста.

Также я заметил, что все предпосылки очень длинные

В этом контесте было три раунда, в связи с чем датасет разбит на 9 частей. Части i-го раунда имеют суффикс _ri.

Я тестировал на 1-м раунде.

Пример объекта:

| Столбец | Описание столбца | Значение |
|---------|------------------|----------|
| premise | Предпосылка     | Charles William Ramsden (11 June 1904 – 16 February 1975) was an English footballer. His regular position was as a forward. <br /> He was born in Bucklow, Cheshire. He played for Rotherham Town, Stockport County, Manchester North End, and Manchester United. |
| hypothesis | Гипотеза | Charles William Ramsden lived for over 70 years. |
| label | Gold label (0 - entailment, 1 - neutral, 2 - contradiction) | 0 |
| reason | Причина, почему такой gold label | Charles William Ramsden (11 June 1904 – 16 February 1975) was an English footballer. He lived more than 70 years.
| uid | ID объекта | 8505cdf9-2f11-45a8-b6a3-02d6044971a7 |

ANLI тоже нужно дообучать

In [ ]:
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--task anli \
--save_path /content/anli_ckpt \
--checkpoint_path '/content/snli_baseline.ckpt' \
--gpus=0,  \
--precision 16 \
--lr=2e-4 \
--batch_size=10 \
--lamb=1.0 \
--workers=1 \
--max_epoch=1 \
--mode train
%cd /content

[Errno 2] No such file or directory: '/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain'
/content
python3: can't open file 'trainer.py': [Errno 2] No such file or directory
/content


In [ ]:
!mkdir /content/anli_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--task anli \
--checkpoint_path '/content/anli_ckpt/last.ckpt' \
--save_path /content/anli_result/ \
--gpus=0, \
--mode eval
%cd /content

/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
Traceback (most recent call last):
  File "trainer.py", line 287, in <module>
    main()
  File "trainer.py", line 278, in main
    evaluate(args)
  File "trainer.py", line 265, in evaluate
    checkpoint = torch.load(args.checkpoint_path, map_location=torch.device('cpu'))
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 571, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 229, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.7/dist-packages/torch/serialization.py", line 210, in __init__
    super(_open_file, self).__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/anli_ckpt/last.ckpt'
/content


## Тестируем модель на [Multi-NLI (MNLI)](https://huggingface.co/datasets/multi_nli)

Есть три набора данных:
- train 
- validation_matched: предложения взяты из того же источника, откуда взяли train => они похожи на train
- validation_mismatched: предложения взяли из другого источника => они сильно отличаются от train.

Пример объекта:

| Столбец | Описание столбца | Значение |
|---------|------------------|----------|
| premise | Предпосылка | harsh weather |
| premise_binary_parse | Дерево зависимостей у предпосылки | ( harsh weather ) | 
| premise_parse | Распарсенная предпосылка через Stanford PCFG Parser 3.5.2 | (ROOT (NP (JJ harsh) (NN weather))) |
| hypothesis | Гипотеза | good weather |
| hypothesis_binary_parse | Дерево зависимостей у гипотезы | ( good ( weather . ) ) |
| hypothesis_parse | Распарсенная гипотеза через Stanford PCFG Parser 3.5.2 | (ROOT (NP (JJ good) (NN weather) (. .))) |
| label | Gold label (0 - entailment, 1 - neutral, 2 - contradiction) | 2 |
| genre | Область, про которую написано в предпосылке и гипотезе | government |
| promptID | ??? | 85161 |
| pairID | ??? | 85161c |

Как обычно, дообучаем модель для MNLI:

In [ ]:
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-large \
--task mnli \
--save_path /content/mnli_ckpt \
--checkpoint_path '/content/snli_baseline.ckpt' \
--gpus=0,  \
--precision 16 \
--lr=2e-5 \
--batch_size=10 \
--lamb=1.0 \
--workers=1 \
--max_epoch=1
%cd /content

/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
Loading checkpoint
Loaded checkpoint
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
Missing logger folder: /content/mnli_ckpt/log
initializing ddp: GLOBAL_RANK: 0, MEMBER: 1/1
Traceback (most recent call last):
  File "trainer.py", line 287, in <module>
    main()
  File "trainer.py", line 276, in main
    train(args)
  File "trainer.py", line 260, in train
    trainer.fit(model)
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py", line 444, in fit
    results = self.accelerator_backend.train()
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/accelerators/ddp_accelerator.py", line 148, in train
    results = self.ddp_train(process_idx=self.task_idx, model=model)
  File "/usr/local/lib/python3.7/dist-packages/pytorch_lightning/accelerators/ddp_accelerator.py", li

In [ ]:
!mkdir /content/mnli_result
%cd /content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
!python trainer.py \
--bert_path /content/roberta-larg \
--task mnli \
--checkpoint_path '/content/mnli_ckpt/last.ckpt' \
--save_path /content/mnli_result/ \
--gpus=0, \
--mode eval
%cd /content

/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 373, in get_config_dict
    raise EnvironmentError
OSError

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "trainer.py", line 287, in <module>
    main()
  File "trainer.py", line 278, in main
    evaluate(args)
  File "trainer.py", line 264, in evaluate
    model = ExplainNLP(args)
  File "trainer.py", line 50, in __init__
    self.model = ExplainableModel(self.bert_dir)
  File "/content/logical_reasoning/Self_Explaining_Structures_Improve_NLP_Models/explain/model.py", line 21, in __init__
    self.bert_config = RobertaConfig.from_pretrained(bert_dir, output_hidden_states=False)
  File "/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py", line 329, in from_pretrained
    config_dict, kwargs = cls

# Итог


| Датасет | test_acc | test_loss | val_loss | Место среди моделей | Где смотрел модели? |
|---------|----------|-----------|----------|-----|----|
| SNLI    | 0.9173   | -0.7510   | -0.7510  | 5 из 91 | [Papers With Code](https://paperswithcode.com/sota/natural-language-inference-on-snli) |
| SICK   | 0.8849   | -0.7164   | -0.7164  | 2 из 12 | [Результаты в статье про NeuralLog](https://user-images.githubusercontent.com/29106126/144738746-74316acb-06b6-4f98-b451-5f6e0e5d9dc7.png) |
| ANLI_r1 | 0.3880   |  0.6372   |  0.6372  | 7 из 8 | [Leaderboard на гитхабе датасета](https://github.com/facebookresearch/anli) |
| MNLI | 0.8046 | -0.4656 | -0.4656 | 29 из 37 | [Papers With Code](https://paperswithcode.com/sota/natural-language-inference-on-multinli)